# Predicting GBP/USD Currency Exchange Rate Using RNN (POC 4)
## Step 1: Data Loading and Preprocessing


The historical data contains information about the volume traded and the corresponding GBP/USD exchange rate at different timestamps.

1) Import necessary libraries including numpy, pandas, scikit-learn, and Keras.
2) Load Data: Load the historical data from a CSV file into a pandas DataFrame.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam

# get the historical data of GBPUSD from 2022.01.01 to 2023.12.31
# Load the ask dataset
ask_data = pd.read_csv('GBPUSD_5 Mins_Ask_2022.01.01_2023.12.31.csv')

# Load the bid dataset
bid_data = pd.read_csv('GBPUSD_5 Mins_Bid_2022.01.01_2023.12.31.csv')

# Ensure that both datasets are aligned by time before merging
merged_data = pd.merge(ask_data, bid_data, on='Time (EET)', suffixes=('_ask', '_bid'))

print(merged_data.shape)

We’ll predict the **closing prices,** which can be accessed in the **Close column.**



In [ ]:
# Selecting features and target
features = merged_data[['Open_ask', 'Volume_ask', 'Volume_bid']].values
target = merged_data['Close_ask'].values

# Normalizing the data
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)
target_scaled = scaler.fit_transform(target.reshape(-1, 1))

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target_scaled, test_size=0.2, random_state=42)

# Reshaping input to be [samples, time steps, features] which is required for LSTM
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))


## Step 2: Model Building

In [ ]:
# Building the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(1, 3)))  # Adjusting the input_shape to account for three features
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

print(model.summary()) 


## Step 3: Model Training

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

## Step 4: Model Evaluation

In [ ]:
# Predicting using the trained model
predictions = model.predict(X_test)

# Rescaling the predicted values back to original scale
predictions_rescaled = scaler.inverse_transform(predictions)
y_test_rescaled = scaler.inverse_transform(y_test)

# Calculating the mean squared error and mean absolute percentage error
mse = mean_squared_error(y_test_rescaled, predictions_rescaled)
mape = mean_absolute_percentage_error(y_test_rescaled, predictions_rescaled)

# Creating a table for results
results_df = pd.DataFrame({
    "Metric": ["Mean Squared Error", "Mean Absolute Percentage Error"],
    "Value": [mse, mape]
})

# Displaying the results in a DataFrame
print(results_df)